# CV Evaluation System

In the hiring process, recruiters and HR professionals often need to evaluate a large number of resumes to identify the most suitable candidates for a given job position. This process is time-consuming, requiring manual review of resumes against the job description, and may introduce human biases.

In [13]:
# Install the required Packages / Libraries
%%capture --no-stderr
%pip install langchain langchain_google_genai PyMuPDF --quiet

In [14]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from google.colab import userdata
from langchain_core.runnables import RunnablePassthrough

In [15]:
template = """
You are an HR assistant. Evaluate the resume based on the job description.
Job Description: {job_description}
Resume: {resume}

Provide a score from 1 to 10 and give a short reason for the score.
"""

prompt = PromptTemplate(input_variables=["job_description", "resume"], template=template)

In [16]:
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash",
                             verbose=True,
                             temperature=0.0,
                             google_api_key=userdata.get("GOOGLE_API_KEY"))

In [17]:
job_description = """
We are looking for a software engineer with experience in Python, machine learning, and cloud computing.
The ideal candidate will have strong problem-solving skills and the ability to work in a team environment.
"""

# Prepare the Chain



In [18]:
chain = (
             {"job_description": RunnablePassthrough(), "resume": RunnablePassthrough()}
             | prompt
        | llm
    )

uses RunnablePassthrough() to pass the input and output between the prompt and the model, creating a more flexible chain.

In [19]:
import fitz  # PyMuPDF

# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

In [26]:
#pdf_path = "/content/resume.pdf"
pdf_path="/content/sample-cv.pdf"
pdf_path2="/content/sample-cv2.pdf"

In [27]:
resume_text = extract_text_from_pdf(pdf_path2)

In [28]:
resume_text

'Jane Doe \nCity, State 1 (123) 456-78901@gmail.com \nEDUCATION \nUniversity \nCity, State \nBachelor of Science in Computer Information Technology \nAugust 2019 \nEXPERIENCE \nCompany \nCity, State \nTechnology Business Systems Associate \nJune 2021 - June 2023 \n• \nTrack vulnerabilities and vulnerability patches for 200+ Windows and Linux based servers using Jira. \n• \nIncorporate strategies with 5+ engineers to coordinate the resolution of vulnerabilities to avoid further escalation. \n• \nCreate reports using PowerBI to conduct trend analysis for recurring vulnerabilities. \n• \nCoordinate project plans, schedules and assist with IT technical writing in Confluence. \n• \nAssist with maintenance of information security policies, standards, guidelines based on compliance requirements. \nCompany \nCity, State \nLoan Documentation Processor \nMay 2020 — June 2021 \n• \nObtained and verified copies of loan applicants credit histories, financial and employment information. \n• \nLead v

In [29]:
chain.invoke({"job_description": job_description, "resume": resume_text}).content


"Score: 6/10\n\nReason: The resume shows some potential, particularly with the listed Python skill and Azure Fundamentals certification. The Technology Business Systems Associate role also hints at relevant experience. However, the resume lacks explicit examples of machine learning experience and doesn't showcase the depth of Python or cloud computing skills needed for a Software Engineer role. The other roles are less relevant. More detail and quantifiable achievements in relevant areas would significantly improve the resume."